In [1]:
#Test random sampling
import sys
sys.path.append("/Users/ingenia/git/instrumento/")
sys.path.append("/Users/ingenia/git/utilityFuncs/")
import random_sampling
import instrumento as ins
import numpy as np
import pandas as pd

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix as cm

import query_by_committee_sampling
import density_sampling
import random_sampling
import uncertainty_sampling

from sklearn.cross_validation import train_test_split
from sklearn.grid_search import (GridSearchCV, RandomizedSearchCV)

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.lda import LDA
from sklearn.qda import QDA
from sklearn.linear_model import LogisticRegression


log=ins.instrumento(filename="./benLogFile.txt",printout=True)

log.act("imported modules and starting")

./benLogFile.txt

activity,imported modules and starting,Oct-29-02:30:51PM


In [10]:
log.act("reading data")
path="/Users/ingenia/git/data/data_sampling/"
dataFilename=path+"test_data.tsv"
colnames = ["ID", "bot", "f1", "f2", "f3", "f4", "f5", "f6", "f7", "f8", "f9", "f10", "f11", "f12", "f13", "f14", "f15", "f16", "f17", "f18", "f19", "f20", "f21", "f22", "f23", "f24", "f25", "f26"]
df = pd.read_csv(dataFilename, sep="\t", index_col=0, header=None, names=colnames)
df.head()


activity,reading data,Oct-29-02:41:50PM


,bot,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26
ID,,,,,,,,,,,,,,,,,,,,,
808354,0,5.47,0.21,2.11,14.50,1.34,6.09,11.90,0.42,4.90,...,9.44,732,32,17.23,33.17,0.64,0.48,522674.19,1456515.46,1195630.64
433328,1,0.94,0.81,5.66,17.72,4.66,23.30,0.82,0.78,5.09,...,6.91,6632125,483175,6.69,40.63,0.93,0.26,4483198.99,10798649.88,7202.57
1146111,0,7.33,1.85,3.22,36.06,11.96,27.02,10.13,3.67,5.85,...,9.06,26036,797,19.66,97.39,0.02,0.14,791116.41,2903179.75,115239.53
861745,0,3.22,10.11,56.72,11.88,63.25,436.92,0.83,0.21,0.14,...,7.79,6585,82,1.17,5.06,0.41,0.49,1171639.03,2396799.99,5224761.29
306411,0,0.50,0.00,1.50,0.87,0.00,0.87,1.00,0.00,1.00,...,4.49,8,3,2.00,2.00,0.50,0.50,527814.25,787291.59,3157474.20


In [16]:
log.act("Start")
unique_ids = len(df.index.unique())
shape = df.shape
log.act("Data read")
log.sum(str(shape), "Shape")
log.sum(str(unique_ids), "Unique Rows")
log.sum(str(df.bot.mean()), "Mean bot percent")
log.act("End")


activity,Start,Oct-29-02:58:41PM

activity,Data read,Oct-29-02:58:41PM

summary,(2222978, 27) Shape,Oct-29-02:58:41PM

summary,2222978 Unique Rows,Oct-29-02:58:41PM

summary,0.000243817077812 Mean bot percent,Oct-29-02:58:41PM

activity,End,Oct-29-02:58:41PM


In [35]:
log.act("Start")
log.act("Starting with a random sample since it's most straitfwd and I can use the sklearn tools I'm familiar with")
log.act("The very unbalanced (only 542 bots) labels are a little concerning")
random_train, random_valid = train_test_split(df, test_size=0.9, random_state=42)
train_bots = np.sum(random_train['bot'])
log.sum(str(train_bots), "bots in the train set (out of 542)")
log.act("End")


activity,Start,Oct-29-03:28:42PM

activity,Starting with a random sample since it's most straitfwd and I can use the sklearn tools I'm familiar with,Oct-29-03:28:42PM

activity,The very unbalanced (only 542 bots) labels are a little concerning,Oct-29-03:28:42PM

summary,63.0 bots in the train set (out of 542),Oct-29-03:28:45PM

activity,End,Oct-29-03:28:45PM


In [36]:
log.act("Start")
log.act("Try defining some rudimentary classifiers to train and apply some very basic tuning")
clf1 = LogisticRegression()
grid1 = {'C': np.logspace(-3,3, 3)}
clf2 = RandomForestClassifier(n_estimators=5)
grid2 = {'max_features':['sqrt', 0.5], 'max_depth': [15, 5], 'min_samples_split': [4,9,25], 'min_samples_leaf':[1,3,5]}
log.act("End")


activity,Start,Oct-29-03:29:07PM

activity,Try defining some rudimentary classifiers to train and apply some very basic tuning,Oct-29-03:29:07PM

activity,End,Oct-29-03:29:07PM


In [38]:
log.act("Start")
log.act("Running a CV on 10% data to find best logistic regression params...now I could run an uncertainty sampling for example, with maybe reasonable params")
grid = GridSearchCV(clf1, grid1, cv=3, scoring='f1')
%time grid.fit(random_train.drop(['bot'], axis=1), random_train['bot']) 
print "Here are the scores for each parameter:"
log.act("End")
grid.grid_scores_


activity,Start,Oct-29-03:32:49PM

activity,Running a CV on 10% data to find best logistic regression params...now I could run an uncertainty sampling for example, with maybe reasonable params,Oct-29-03:32:49PM
CPU times: user 59.7 s, sys: 1.18 s, total: 1min
Wall time: 1min 2s
Here are the scores for each parameter:

activity,End,Oct-29-03:33:51PM


[mean: 0.03221, std: 0.03712, params: {'C': 0.001},
 mean: 0.03891, std: 0.03467, params: {'C': 1.0},
 mean: 0.03192, std: 0.03671, params: {'C': 1000.0}]

In [39]:
clf1.set_params(**grid.best_params_)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [42]:
log.act("Start")
log.act("Running a CV on 10% data to find best RF params, after re-training this at least I could get 3 best features")
grid = RandomizedSearchCV(clf2, grid2, n_iter=4, cv=3, scoring="f1")
%time grid.fit(random_train.drop(['bot'], axis=1), random_train['bot']) 
print "Here are the scores for each parameter:"
log.act("End")
clf2.set_params(**grid.best_params_)
grid.grid_scores_


activity,Start,Oct-29-03:45:32PM

activity,Running a CV on 10% data to find best RF params, after re-training this at least I could get 3 best features,Oct-29-03:45:32PM
CPU times: user 25.2 s, sys: 766 ms, total: 26 s
Wall time: 26.3 s
Here are the scores for each parameter:

activity,End,Oct-29-03:45:58PM


[mean: 0.32214, std: 0.09452, params: {'max_features': 0.5, 'min_samples_split': 25, 'max_depth': 15, 'min_samples_leaf': 3},
 mean: 0.40677, std: 0.10414, params: {'max_features': 0.5, 'min_samples_split': 9, 'max_depth': 15, 'min_samples_leaf': 3},
 mean: 0.19222, std: 0.04095, params: {'max_features': 'sqrt', 'min_samples_split': 4, 'max_depth': 5, 'min_samples_leaf': 1},
 mean: 0.47778, std: 0.16405, params: {'max_features': 0.5, 'min_samples_split': 25, 'max_depth': 15, 'min_samples_leaf': 5}]

In [51]:
log.act("Start")
log.act("Retrain on bigger portion of data, to get 3 most important features")
clf2.n_estimators = 10
random_sample, extra = train_test_split(random_valid, test_size=0.60, random_state=42)
print random_sample.shape
%time clf2.fit(random_sample.drop(['bot'], axis=1), random_sample['bot']) 
log.act("End")


activity,Start,Oct-29-03:59:09PM

activity,Retrain on bigger portion of data, to get 3 most important features,Oct-29-03:59:09PM
(800272, 27)
CPU times: user 1min 46s, sys: 1.35 s, total: 1min 48s
Wall time: 1min 51s

activity,End,Oct-29-04:01:03PM


In [54]:
log.act("Start")
pd.Series(index=random_sample.drop(['bot'], axis=1).columns, data = clf2.feature_importances_).order().plot(kind='barh', title='Variable Importances')
log.act("End")


activity,Start,Oct-29-04:01:22PM

activity,End,Oct-29-04:01:22PM


In [55]:
pd.Series(index=random_sample.drop(['bot'], axis=1).columns, data = clf2.feature_importances_).order()

f16    0.002463
f6     0.005863
f1     0.008465
f7     0.008942
f24    0.009253
f13    0.009854
f3     0.010767
f17    0.011044
f26    0.013212
f2     0.014377
f25    0.014793
f10    0.014966
f15    0.015638
f5     0.018904
f12    0.020390
f8     0.022597
f14    0.023991
f18    0.033845
f11    0.035666
f4     0.042900
f20    0.049888
f9     0.050497
f21    0.054940
f23    0.101986
f22    0.137193
f19    0.267564
dtype: float64

In [ ]:
vals = uncertainty_sampling.parallel_reg_uncertainty_sampling(clf1,[dataFilename],[:],1,0.1,8)

In [56]:
?random_sampling.parallel_random_sampling